In [1]:
import pandas as pd

# import data sets that will be used
team_names = pd.read_csv("../csvs/kaggle/predictive/Teams.csv")
reg_season = pd.read_csv("../csvs/kaggle/predictive/RegularSeasonDetailedResults.csv")
pd.set_option('display.max_columns', 50)

# adjust data sets to only 2014 and later
reg_season = reg_season[reg_season.Season > 2013]

In [2]:
'''
splits the regular season by win and loss and will later combine
this is because the dataframe is by game not overall
''' 
reg_season_win = reg_season.groupby(['WTeamID','Season'], as_index = False).agg(sum)
reg_season_lose = reg_season.groupby(['LTeamID','Season'], as_index = False).agg(sum)

# calculates wins and losses for each team
team_wins = reg_season.groupby(['WTeamID','Season']).WTeamID.count()
team_losses = reg_season.groupby(['LTeamID','Season']).LTeamID.count()

# creat lists for wins and losses
xwin = []
for i in team_wins:
    xwin.append(i)

xloss = []
for i in team_losses:
    xloss.append(i)

In [3]:
'''
Reason: Undefeated
Missing 2015 season for teamId=1246 losing data 690
Missing 2014 season for teamId=1455 losing data 1705

Reason: Defeated
Missing 2015 season for teamId=1212 winning data
Missing 2015 season for teamId=1363 winning data 
'''

# inserts a 0 where the value was missing
xloss.insert(691,0)
xloss.insert(1705,0)

xwin.insert(526,0)
xwin.insert(1261,0)

In [4]:
'''
renames columns to make it easier to combine, merges based on team
winning team becomes team and losing team become opponent stats
same but flipped below for the lose regular season
drops unnecessary columns
'''

reg_season_win = reg_season_win.rename(
        columns = {'WTeamID':'TeamID', 'WFGM':'FGM', 'WFGA':'FGA','WFGM3':'FGM3',
                   'WFGA3':'FGA3', 'WFTM':'FTM', 'WFTA':'FTA', 'WOR':'OR', 'WDR':'DR',
                   'WAst':'Ast', 'WTO':'TO', 'WStl':'Stl', 'WBlk':'Blk', 'WPF':'PF',
                   'LFGM':'OFGM', 'LFGA':'OFGA', 'LFGM3':'OFGM3', 'LFGA3':'OFGA3',
                   'LFTM':'OFTM', 'LFTA':'OFTA', 'LOR':'OOR', 'LDR':'ODR',
                   'LAst':'OAst', 'LTO':'OTO', 'LStl':'OStl', 'LBlk':'OBlk', 'LPF':'OPF'})


reg_season_win = reg_season_win.drop(['DayNum','WScore','LTeamID','LScore','NumOT'], axis = 1)


reg_season_lose = reg_season_lose.rename(
        columns={'LTeamID':'TeamID', 'LFGM':'FGM', 'LFGA':'FGA', 'LFGM3':'FGM3',
                 'LFGA3':'FGA3', 'LFTM':'FTM', 'LFTA':'FTA', 'LOR':'OR',
                 'LDR':'DR', 'LAst':'Ast', 'LTO':'TO', 'LStl':'Stl', 'LBlk':'Blk',
                 'LPF':'PF', 'WFGM':'OFGM', 'WFGA':'OFGA', 'WFGM3':'OFGM3',
                 'WFGA3':'OFGA3', 'WFTM':'OFTM', 'WFTA':'OFTA', 'WOR':'OOR', 'WDR':'ODR',
                 'WAst':'OAst', 'WTO':'OTO', 'WStl':'OStl', 'WBlk':'OBlk', 'WPF':'OPF'})


reg_season_lose = reg_season_lose.drop(['DayNum','WScore','WTeamID','LScore','NumOT'], axis = 1)


regular_season = pd.concat([reg_season_win, reg_season_lose])


# reindexs to make it look cleaner
regular_season = regular_season.reindex_axis(reg_season_win.columns, axis = 1)


# Once again takes the sum of every column grouped by each team
regular_season = regular_season.groupby(['TeamID','Season'], as_index = False).agg(sum)

In [5]:
'''
Adds new statistics as columns to the dataset
'''

# creates names array for each team ID 
names = []
for team in regular_season.TeamID:
    names.append(team_names['TeamName'][team_names['TeamID'] == team].values[0])

    
# inserts Team name, wins, losses columns
regular_season.insert(loc=0, column ='TeamName', value = names)
regular_season.insert(loc=3, column = 'W', value = xwin)
regular_season.insert(loc=4, column = 'L', value = xloss)


# creates new columns Games, field goal %, 3-point %, free throw %
regular_season.insert(loc=3, column = 'G', value = regular_season.W + regular_season.L)

# Calculate a teams efficiency
Efficiency = ((regular_season.FGM + regular_season.OR + 
               regular_season.DR + regular_season.Ast +
               regular_season.Stl + regular_season.Blk) - 
              ((regular_season.FGA - regular_season.FGM) + 
            (regular_season.FTA - regular_season.FTM) + regular_season.TO))/regular_season.G

regular_season.insert(loc=6, column = 'Eff', value = Efficiency)

# calculate effective field goal for a team and their opponents
eFG = (regular_season.FGM + .5*regular_season.FGM3)/regular_season.FGA
OeFG = (regular_season.OFGM + .5*regular_season.OFGM3)/regular_season.OFGA

# calculate Turnover Percentage, an estimate of turnovers per 100 plays.
TOV = regular_season.TO/(regular_season.FGA + .44*regular_season.FTA + regular_season.TO)
OTOV = regular_season.OTO/(regular_season.OFGA + .44*regular_season.OFTA + regular_season.OTO)

# calculate offensive and defensive rebounds percentage
ORB = regular_season.OR/(regular_season.OR + regular_season.ODR)
DRB = regular_season.DR/(regular_season.DR + regular_season.OOR)
OORB = regular_season.OOR/(regular_season.OOR + regular_season.DR)
ODRB = regular_season.ODR/(regular_season.ODR + regular_season.OR)

# Simple Stats and percentages
regular_season.insert(loc=8, column = 'FG%', value = regular_season.FGM * 100 / regular_season.FGA)
regular_season.insert(loc=11, column = 'FG3%', value = regular_season.FGM3 * 100 / regular_season.FGA3)
regular_season.insert(loc=14, column = 'FT%', value = regular_season.FTM * 100 / regular_season.FTA)
regular_season.insert(loc=25, column = 'OFG%', value = regular_season.OFGM * 100 / regular_season.OFGA)
regular_season.insert(loc=28, column = 'OFG3%', value = regular_season.OFGM3 * 100 / regular_season.OFGA3)
regular_season.insert(loc=31, column = 'OFT%', value = regular_season.OFTM * 100 / regular_season.OFTA)

# Team Stats
regular_season.insert(loc=9, column = 'eFG%', value = 100*eFG)
regular_season.insert(loc=18, column = 'ORB%', value = 100*ORB)
regular_season.insert(loc=20, column = 'DRB%', value = 100*DRB)
regular_season.insert(loc=23, column = 'TOV%', value = 100*TOV)

# Opponent Stats
regular_season.insert(loc=30, column = 'OeFG%', value = 100*OeFG)
regular_season.insert(loc=38, column = 'OORB%', value = 100*OORB)
regular_season.insert(loc=40, column = 'ODRB%', value = 100*ODRB)
regular_season.insert(loc=43, column = 'OTOV%', value = 100*OTOV)


# views new tidy, clean data frame
regular_season.to_csv("regular_season_stats.csv", encoding='utf-8', index=False)
regular_season.head()

,TeamName,TeamID,Season,G,W,L,Eff,FGM,FG%,eFG%,FGA,FGM3,FG3%,FGA3,FTM,FT%,FTA,OR,ORB%,DR,DRB%,Ast,TO,TOV%,Stl,Blk,PF,OFGM,OFGA,OFG%,OeFG%,OFGM3,OFGA3,OFG3%,OFTM,OFTA,OFT%,OOR,OORB%,ODR,ODRB%,OAst,OTO,OTOV%,OStl,OBlk,OPF
0,Abilene Chr,1101,2014,21,2,19,15.714286,427,40.550807,47.198481,1053,140,37.333333,375,332,74.606742,445,168,24.925816,427,66.201550,210,315,20.143241,121,31,453,570,1122,50.802139,56.417112,126,340,37.058824,385,542,71.033210,218,33.798450,506,75.074184,327,255,15.784782,147,105,392
1,Abilene Chr,1101,2015,28,7,21,19.892857,600,40.485830,47.334683,1482,203,37.802607,537,305,72.792363,419,231,24.163180,550,66.185319,332,359,17.725244,166,33,577,701,1424,49.227528,55.301966,173,468,36.965812,437,636,68.710692,281,33.814681,725,75.836820,362,377,18.117683,164,119,463
2,Abilene Chr,1101,2016,27,9,18,26.703704,643,44.162088,50.515110,1456,185,36.345776,509,415,70.698467,587,221,24.501109,608,72.380952,348,362,17.435028,182,66,604,715,1449,49.344375,54.554865,151,409,36.919315,478,674,70.919881,232,27.619048,681,75.498891,347,363,17.215540,138,89,531
3,Abilene Chr,1101,2017,25,9,16,28.240000,611,45.836459,52.475619,1333,177,37.106918,477,298,64.224138,464,189,23.190184,572,68.257757,340,362,19.061059,175,69,554,621,1342,46.274218,52.868852,177,490,36.122449,397,595,66.722689,266,31.742243,626,76.809816,340,370,18.745567,151,79,431
4,Abilene Chr,1101,2018,27,12,15,31.555556,689,45.180328,50.918033,1525,175,32.467532,539,355,70.436508,504,244,26.521739,627,70.055866,375,389,18.213657,193,98,568,669,1466,45.634379,51.739427,179,533,33.583490,449,633,70.932070,268,29.944134,676,73.478261,325,418,19.329301,176,79,477
